In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
from scipy import interpolate
from datetime import datetime

In [3]:
# Deterministic
lowquality = 0             # 0 = High, 1 = Low
y_f = 1
g = 9.81
fetch = [2392,3128,3480,2992,1622,738,240,120,120,176,750,2002,2848,3084,2574,1762]
angle = [10.5,33,55.5,78,100.5,123,145.5,168,-169.5,-147,-124.5,-102,-79.5,-57,-34.5,-12]
winddir = ["NNE", "NE", "ENE", "E", "ESE", "SE", "SSE", "S", "SSW", "SW", "WSW", "W", "WNW", "NW", "NNW", "N"]
revetment = [[225,250],[100,120],[70,80],[40,50]]

In [4]:
# Load distributions
# Water level
data = np.loadtxt('Cumulative_density_function_water_level.txt', delimiter=' ', unpack=True)

# Wind
df = pd.read_excel (r'exc_wind.xlsx')
exc_vel = df["Wind velocity [m/s] "].tolist()
exc_wind = []

for i in range(16):
    exc_wind.append(1 - np.array(df[winddir[i]].tolist()))

In [8]:
ch = np.arange(8.3, 8.61, 0.1)
iterations = len(ch)
n = 2000000

Pf = []

for k in range(iterations):
    crest_height = ch[k]
    fail = 0

    for i in range(n):    
        # Generate a random wind direction
        j = random.choices([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],[6.4,10.4,14.5,10.3,1.9,2.3,2.2,2.8,4.5,4.9,4.1,2.2,13.9,10.5,6.0,3.1])[0]

        # Generate a random wind speed
        w = random.random()
        u = np.interp(w, exc_wind[j], exc_vel)

        # Calculate y_beta
        y_beta = 1 - 0.0033 * np.absolute(angle[j])
        if(np.absolute(angle[j]) > 80):
            y_beta = 1 - 0.0033 * 80

        # Generate water height, Rc and d
        #w = random.random();
        b = random.normalvariate(2.35, 0.3);
        swl = np.interp(w, data[1], data[0])
        Rc = crest_height - swl
        d = swl - b

        # If there is no water depth, there is no failure
        if(d < 0):
            continue

        # Calculate wave height
        F = (fetch[j] * g) / u**2
        D = (d * g) / u**2
        Hdak = 0.283 * np.tanh(0.53 * D**0.75) * np.tanh((0.0125 * F**0.42)/(np.tanh(0.53 * D**0.75)))
        H = (Hdak * u**2) / g

        # Angle checks and reductions
        if(np.absolute(angle[j]) > 110):
            H = 0
        elif(np.absolute(angle[j]) > 80):
            H = H * (110 - np.absolute(angle[j])) / 30
            if(H < 0):
                H = 0

        # Grass cover quality
        if(H > 2):
            wavecat = 2
        elif(H > 1):
            wavecat = 1
        else:
            wavecat = 0
        index = lowquality+wavecat
        sigma = (np.log(1 + (revetment[index][1]/revetment[index][0])**2))**0.5
        mu = np.log(revetment[index][0]) - 0.5 * sigma**2
        q = np.random.lognormal(mu, sigma);
        
        # Check for overflow, if there is overflow, count it as a fail and continue the loop
        if(Rc < 0):
            qov = 0.54 * ((9.81 * np.absolute(Rc ** 3)) ** 0.5) * (10 ** 3)
            if(qov > q):
                fail += 1
            continue

        # Calculate exponents
        a = 0.09;
        b = 1.5;

        # Z = R - S
        Z = 1
        if(H > 0):
            Z = q - 10**3 * a * np.exp(-(((b * Rc) / (H * y_f * y_beta))**1.3)) * ((g * H**3)**0.5)

        if(Z < 0):
            fail += 1

    Pf.append(fail / n)
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print(str(crest_height) + "|" + str(current_time) + "|" + str(fail) + "|" + str(Pf[k]) + "|" + str(round(1/Pf[k],2)))

8.3|15:02:37|81|4.05e-05|24691.36
8.4|15:05:05|53|2.65e-05|37735.85
8.5|15:07:32|48|2.4e-05|41666.67
8.6|15:10:00|33|1.65e-05|60606.06
